In [210]:
import pandas as pd
import numpy as np
from sklearn.cluster import MiniBatchKMeans
from sklearn.ensemble import RandomForestRegressor

In [221]:
filename = 'data/201910-bluebikes-tripdata-weather-holiday'

trips_df = pd.read_csv(filename + '.csv')
trips_df.index.name = 'index'

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [222]:
trips_df_cut = trips_df[['weekend', 'holiday', 'HourlyDryBulbTemperature','HourlyPrecipitation','HourlyStationPressure','HourlyWindSpeed']]
trips_df_cut.weekend = trips_df_cut.weekend.astype(int)
trips_df_cut.holiday = trips_df_cut.holiday.astype(int)
trips_df_cut = trips_df_cut.fillna(method='ffill')
null_columns=trips_df_cut.columns[trips_df_cut.isnull().any()]
trips_df_cut[trips_df_cut.isnull().any(axis=1)][null_columns]
trips_arr = trips_df_cut.values

In [223]:
kmeans = MiniBatchKMeans(n_clusters=10)
trips_arr_distances = kmeans.fit_transform(trips_arr)

In [233]:
trips_df_cut['starttime'] = pd.to_datetime(trips_df['starttime'])
trips_df_cut_indexed = trips_df_cut.set_index(pd.DatetimeIndex(trips_df_cut['starttime']))

trips_grouped = trips_df_cut_indexed.groupby(pd.Grouper(key='starttime', freq='D'))
trips_agg = trips_grouped.agg('mean')
trips_count_arr = np.array(trips_grouped.size())
# trips_agg
trips_agg_arr = trips_agg.values
# print(trips_count_arr)
# trips_agg_arr

In [234]:
regr = RandomForestRegressor()
regr.fit(trips_agg_arr, trips_count_arr)

/usr/local/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [235]:
test_pt = [[0, 0, 0, 0, 0, 0]]


[8876.2]


In [239]:
number_of_trips = int(regr.predict(test_pt)[0])
print(number_of_trips)
dists = np.reciprocal(kmeans.transform(test_pt)[0])
weights = np.copy(kmeans.labels_)
for i in range(len(dists)):
    weights = np.where(kmeans.labels_==i, dists[i], weights)
weights
trips_df.sample(n=number_of_trips, weights=weights)

11448


,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyPrecipitation,HourlyRelativeHumidity,HourlyStationPressure,HourlyVisibility,HourlyWindDirection,HourlyWindSpeed,bikeid,birth year,...,start station id,start station latitude,start station longitude,start station name,starttime,stoptime,tripduration,usertype,holiday,weekend
index,,,,,,,,,,,,,,,,,,,,,
31973,44.0,53.0,0.0,72.0,30.04,10.0,150,8.0,3759,2000,...,10,42.350406,-71.108279,B.U. Central - 725 Comm. Ave.,2019-10-03 20:43:35.039,2019-10-03 20:49:35.033,359,Subscriber,False,False
141589,36.0,61.0,0.0,39.0,30.05,10.0,0,0.0,3704,1971,...,107,42.362500,-71.088220,Ames St at Main St,2019-10-15 14:07:11.670,2019-10-15 14:27:40.652,1228,Customer,False,False
221432,54.0,59.0,0.0,83.0,29.74,10.0,290,11.0,6007,1975,...,327,42.374878,-71.063835,Main St at Thompson Sq,2019-10-23 08:13:27.536,2019-10-23 08:29:09.519,941,Subscriber,False,False
45094,31.0,52.0,0.0,45.0,30.42,10.0,350,7.0,4968,1993,...,407,42.388289,-71.110715,Craigie at Summer St,2019-10-05 08:43:05.095,2019-10-05 08:48:11.524,306,Subscriber,False,True
136884,37.0,52.0,0.0,57.0,30.09,10.0,300,10.0,4992,1988,...,14,42.337417,-71.102861,HMS/HSPH - Avenue Louis Pasteur at Longwood Ave,2019-10-15 06:58:16.176,2019-10-15 07:01:02.532,166,Subscriber,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157344,52.0,59.0,0.0,78.0,29.67,10.0,100,21.0,3697,1981,...,381,42.374384,-71.100157,Inman Square at Springfield St.,2019-10-16 18:22:24.907,2019-10-16 18:35:44.799,799,Subscriber,False,False
171148,39.0,56.0,NaN,53.0,NaN,10.0,280,18.0,3285,1998,...,126,42.315692,-71.098634,Egleston Square - Atherton St at Washington St,2019-10-18 09:42:49.327,2019-10-18 09:58:25.166,935,Subscriber,False,False
250272,49.0,57.0,0.0,74.0,30.16,10.0,80,8.0,2636,2000,...,12,42.336244,-71.087986,Ruggles T Stop - Columbus Ave at Melnea Cass Blvd,2019-10-25 16:21:50.928,2019-10-25 16:34:22.117,751,Subscriber,False,False
